In [13]:
import requests
import re
from bs4 import BeautifulSoup
from tqdm import tqdm

import gzip
import json
import codecs
from multiprocessing.dummy import Pool, Queue

In [11]:
mas_of_urls=[]
for n in tqdm(range(1,14)):
    soup = BeautifulSoup(requests.get('https://gg.deals/games/?type=1&page='+str(n)).text, 'lxml')
    mas_of_candidats=soup.find_all('a', class_="main-image")
    for cand in mas_of_candidats:
        tek_url='https://gg.deals' + str(cand['href'])
        mas_of_urls.append(tek_url)
    
print(len(mas_of_urls))
mas_of_urls=mas_of_urls[0:300]
print(len(mas_of_urls))

100%|██████████| 13/13 [00:10<00:00,  1.28it/s]

312
300


In [91]:
def get_user_score(soup):
    otv=soup.find('a', class_="score-circle score-userscore")
    if otv is not None:
        otv=otv.find("span",class_="overlay")
        if otv is not None:
            otv=float(otv.text)
    return otv

def get_review_label(soup):
    otv=soup.find('span', class_="reviews-label")
    if otv is not None:
        otv=re.sub(r'[\d (),]','',otv.text)
    return otv

def get_review_positive_pctg(soup):
    otv=soup.find('span', class_="reviews-label")
    if otv is not None:
        otv=int(otv['title'][0:2])
    return otv

def get_review_count(soup):
    otv=soup.find('span', class_="reviews-label")
    if otv is not None:
        otv=otv.find("span",class_="semi-transparent")
        if otv is not None:
            otv=int(re.sub(r'[,-.]', '',otv.text.strip().strip("()")))
    return otv


def get_dlcs(soup):
    tmp=soup.find('section', id='game-dlcs')
    if tmp is not None:
        tmp=tmp.find_all('a', attrs={'class':'full-link', 'href':True})
    otv=[x['href'] for x in tmp]
    link_more=soup.find('div', class_='list-show-more')
    if link_more is not None:
        link_more=link_more.find('a')['data-url']
        if link_more is not None:
            headers={'x-requested-with':'XMLHttpRequest'}
            dop_soup=BeautifulSoup(requests.get('https://gg.deals' + link_more,headers=headers).text, 'lxml')
            dop_tmp=dop_soup.find_all('a', attrs={'class':'full-link', 'href':True})
            for x in dop_tmp:
                otv.append(x['href'])
    return otv
    


def process_page(url):
    soup=BeautifulSoup(requests.get(url).text, 'lxml')
    my_dict={}
    
    my_dict['url']=url
    my_dict['name']=soup.find_all('span', itemprop='name')[3].text
    
    my_dict['img']=soup.find('img',class_="image-game")['src']
    my_dict['market_url']=requests.get(soup.find_all('a',class_="game-link-widget")[0]['href']).url
    
    my_dict['wishlist_count']=int(soup.find_all('span',class_="user-count")[0].find("span",class_='count').text)
    my_dict['alert_count']=int(soup.find_all('span',class_="user-count")[2].find("span",class_='count').text)
    my_dict['owners_count']=int(soup.find_all('span',class_="user-count")[4].find("span",class_='count').text)
    
    my_dict['release_date']=soup.find('div', class_="game-info-details-section game-info-details-section-release").find("p", class_="game-info-details-content").text
    my_dict['developer']=soup.find('div', class_="game-info-details-section game-info-details-section-developer").find("p", class_="game-info-details-content").text
    my_dict['metacritic_score']=int(soup.find('a', class_="score-circle score-metascore").find("span",class_="overlay").text)
    
    if get_user_score(soup)is not None:
        my_dict['user_score']=get_user_score(soup)
    if get_review_label(soup) is not None:
        my_dict['review_label']=get_review_label(soup)
    if get_review_positive_pctg(soup) is not None:
        my_dict['review_positive_pctg']=get_review_positive_pctg(soup)
    if get_review_count(soup) is not None:
        my_dict['review_count']=get_review_count(soup)
    
    my_dict['genres']=[x.text for x in soup.find('div', id='game-info-genres').find_all('a', class_='badge')]
    my_dict['tags']=[x.text for x in soup.find('div', id='game-info-tags').find_all('a', class_='badge')]
    my_dict['features']=[x.text for x in soup.find('div', id='game-info-features').find_all('a', class_='badge')]
    
    my_dict['pc_systems']=[x.text for x in soup.find('div', class_='game-requirements-tabs').find_all('a')]
    
    if get_dlcs(soup) is not None:
        my_dict['dlcs']=get_dlcs(soup)
    
    
    return my_dict

In [92]:
mm=process_page(mas_of_urls[0])
mm

{'url': 'https://gg.deals/game/nier-replicant-ver122474487139/',
 'name': 'NieR Replicant ver.1.22474487139...',
 'img': 'https://img.gg.deals/24/a3/f2faeb72a2615d368c87c8bbb0e328e00532_307xt176.jpg',
 'market_url': 'https://store.steampowered.com/app/1113560/',
 'wishlist_count': 3521,
 'alert_count': 387,
 'owners_count': 245,
 'release_date': '23 Apr 2021',
 'developer': 'Square Enix, Toylogic Inc.',
 'metacritic_score': 79,
 'genres': ['Action', 'Adventure', 'RPG'],
 'tags': ['Action',
  'Action RPG',
  'Adventure',
  'Anime',
  'Atmospheric',
  'Bullet Hell',
  'Character Action Game',
  'Cult Classic',
  'Dark Fantasy',
  'Dark Humor',
  'Great Soundtrack',
  'Hack and Slash',
  'JRPG',
  'Magic',
  'Nudity',
  'Post-apocalyptic',
  'RPG',
  'Singleplayer',
  'Story Rich',
  'Violent'],
 'features': ['Partial Controller Support',
  'Single-player',
  'Steam Achievements'],
 'pc_systems': ['Windows'],
 'dlcs': ['/ru/dlc/nier-replicant-4-yorha/']}

In [73]:
queue = Queue()
for el in mas_of_urls:
    queue.put(el)  # очередь ссылок на книги


def process_page_wrapper(nomer_potoka):
    with gzip.open('data/part_{:05d}.jsonl.gz'.format(nomer_potoka), mode='wb') as f_json:
        f_json = codecs.getwriter('utf8')(f_json)

        while not queue.empty():
            record = process_page(queue.get())
            print(record)
            record_str = json.dumps(record, ensure_ascii=False)
            print(record_str, file=f_json)

            # счетчик должен атомарно обновиться
            with lock:
                pbar.update(1)


with Pool(processes=2) as pool, tqdm(total=queue.qsize()) as pbar:
    lock = pbar.get_lock()
    pool.map(process_page_wrapper, range(pool._processes))

  1%|          | 2/300 [00:01<05:29,  1.11s/it]

{'url': 'https://gg.deals/game/nier-replicant-ver122474487139/', 'name': 'NieR Replicant ver.1.22474487139...', 'img': 'https://img.gg.deals/24/a3/f2faeb72a2615d368c87c8bbb0e328e00532_307xt176.jpg', 'market_url': ('https://store.steampowered.com/app/1113560/',), 'wishlist_count': (3521,), 'alert_count': (386,), 'owners_count': (245,), 'release_date': ('23 Apr 2021',), 'developer': ('Square Enix, Toylogic Inc.',), 'metacritic_score': (79,), 'genres': (['Action', 'Adventure', 'RPG'],), 'tags': (['Action', 'Action RPG', 'Adventure', 'Anime', 'Atmospheric', 'Bullet Hell', 'Character Action Game', 'Cult Classic', 'Dark Fantasy', 'Dark Humor', 'Great Soundtrack', 'Hack and Slash', 'JRPG', 'Magic', 'Nudity', 'Post-apocalyptic', 'RPG', 'Singleplayer', 'Story Rich', 'Violent'],), 'features': (['Partial Controller Support', 'Single-player', 'Steam Achievements'],), 'pc_systems': ['Windows']}
{'url': 'https://gg.deals/game/nioh-complete-edition-complete-edition/', 'name': 'Nioh: Complete Edition 

  1%|▏         | 4/300 [00:03<04:30,  1.09it/s]

{'url': 'https://gg.deals/game/styx-master-of-shadows/', 'name': 'Styx: Master of Shadows', 'img': 'https://img.gg.deals/ea/8d/c68bb3c3af1fac80de73a542f4ce73b1de88_307xt176.jpg', 'market_url': ('https://store.steampowered.com/app/242640/',), 'wishlist_count': (948,), 'alert_count': (76,), 'owners_count': (7611,), 'release_date': ('07 Oct 2014',), 'developer': ('Cyanide Studio',), 'metacritic_score': (71,), 'genres': (['Action'],), 'tags': (['Stealth', 'Fantasy', 'Singleplayer', 'Action', 'Adventure', 'Third Person', 'RPG', 'Assassin', 'Parkour', 'Steampunk', 'Dark Fantasy', 'Atmospheric', 'Magic', 'Story Rich', 'Difficult', 'Open World', 'Great Soundtrack', 'Gore', 'Indie', 'Multiplayer'],), 'features': (['Single-player', 'Steam Achievements', 'Full controller support', 'Steam Trading Cards', 'Steam Cloud', 'Remote Play on TV'],), 'pc_systems': ['Windows']}
{'url': 'https://gg.deals/game/death-stranding/', 'name': 'DEATH STRANDING', 'img': 'https://img.gg.deals/40/11/e12ed68a19f912ada6

  2%|▏         | 5/300 [00:04<05:20,  1.09s/it]

...

  4%|▍         | 12/300 [00:14<06:35,  1.37s/it]

{'url': 'https://gg.deals/game/nioh-2-the-complete-edition/', 'name': 'Nioh 2 - The Complete Edition', 'img': 'https://img.gg.deals/d2/05/48755cc9166cca7dc08ec73ceff46e2d9e86_307xt176.jpg', 'market_url': ('https://store.steampowered.com/app/1325200/',), 'wishlist_count': (2725,), 'alert_count': (360,), 'owners_count': (524,), 'release_date': ('05 Feb 2021',), 'developer': ('KOEI TECMO GAMES CO., LTD.',), 'metacritic_score': (86,), 'genres': (['Action', 'RPG'],), 'tags': (['RPG', 'Action', 'Souls-like', 'JRPG', 'Difficult', 'Co-op', 'Fantasy', 'Multiplayer', 'Story Rich', 'Violent', 'Historical', 'Dark Fantasy', 'Character Customization', 'Singleplayer', 'Dark', 'Medieval', 'Atmospheric', 'Hack and Slash', 'Gore', 'Female Protagonist'],), 'features': (['Single-player', 'Online Co-op', 'Steam Achievements', 'Steam Trading Cards', 'Partial Controller Support', 'Steam Cloud', 'Stats', 'Steam Leaderboards'],), 'pc_systems': ['Windows']}


  4%|▍         | 13/300 [00:15<06:35,  1.38s/it]

{'url': 'https://gg.deals/game/marvels-avengers/', 'name': "Marvel's Avengers", 'img': 'https://img.gg.deals/50/f8/3f1cd008964ed9addf603245b6998e7a3bf9_307xt176.jpg', 'market_url': ('https://store.steampowered.com/app/997070/',), 'wishlist_count': (1975,), 'alert_count': (225,), 'owners_count': (552,), 'release_date': ('04 Sep 2020',), 'developer': ('Crystal Dynamics, Eidos-Montréal, Nixxes',), 'metacritic_score': (68,), 'genres': (['Action', 'Adventure', 'RPG'],), 'tags': (['Action', 'Adventure', 'RPG', 'Superhero', 'Multiplayer', 'Online Co-Op', 'Open World', 'Cinematic', 'Singleplayer', 'Comic Book', 'Third Person', 'Loot', 'Sci-fi', 'Co-op', 'PvE', 'Fantasy', 'Team-Based', 'Combat', 'Story Rich', 'Co-op Campaign'],), 'features': (['Single-player', 'Online Co-op', 'Steam Achievements', 'Full controller support', 'Steam Trading Cards', 'In-App Purchases', 'Steam Cloud', 'Remote Play on Tablet'],), 'pc_systems': ['Windows']}


AttributeError: 'NoneType' object has no attribute 'find'

In [40]:
def process_pageOld(url):
    soup=BeautifulSoup(requests.get(url).text, 'lxml')
    my_dict={
            'url':url,
            'name':soup.find_all('span', itemprop='name')[3].text,
        
            'img':soup.find('img',class_="image-game")['src'],
            'market_url':requests.get(soup.find_all('a',class_="game-link-widget")[0]['href']).url,
        
            'wishlist_count':int(soup.find_all('span',class_="user-count")[0].find("span",class_='count').text),
            'alert_count':int(soup.find_all('span',class_="user-count")[2].find("span",class_='count').text),
            'owners_count':int(soup.find_all('span',class_="user-count")[4].find("span",class_='count').text),
        
            'release_date':soup.find('div', class_="game-info-details-section game-info-details-section-release").find("p", class_="game-info-details-content").text,
            'developer':soup.find('div', class_="game-info-details-section game-info-details-section-developer").find("p", class_="game-info-details-content").text,
            'metacritic_score':int(soup.find('a', class_="score-circle score-metascore").find("span",class_="overlay").text),
            #'user_score': float(soup.find('a', class_="score-circle score-userscore").find("span",class_="overlay").text),
            #'review_label':re.sub(r'[\d (),]','', soup.find('span', class_="reviews-label").text),
            #'review_positive_pctg': int(soup.find('span', class_="reviews-label")['title'][0:2]),
            #'review_count':int(re.sub(r'[,-.]', '',soup.find('span', class_="reviews-label").find("span",class_="semi-transparent").text.strip().strip("()"))),
            'genres':[x.text for x in soup.find('div', id='game-info-genres').find_all('a', class_='badge')],
            'tags': [x.text for x in soup.find('div', id='game-info-tags').find_all('a', class_='badge')],
            'features':[x.text for x in soup.find('div', id='game-info-features').find_all('a', class_='badge')],
        
            'pc_systems':[x.text for x in soup.find('div', class_='game-requirements-tabs').find_all('a')],
            'dlcs':['https://gg.deals'+x['href'] for x in soup.find('section', id='game-dlcs').find_all('a', class_='game-info-title')]
            
        
    }
    return my_dict

In [93]:
u=mas_of_urls[0]
print(u)
s=BeautifulSoup(requests.get('https://gg.deals/game/tricky-towers/').text, 'lxml')
get_dlcs(s)

https://gg.deals/game/nier-replicant-ver122474487139/


['/ru/dlc/tricky-towers-indie-friends-pack/',
 '/ru/dlc/tricky-towers-spirit-animal-pack/',
 '/ru/dlc/tricky-towers-galaxy-bricks/',
 '/ru/dlc/tricky-towers-candy-bricks/',
 '/ru/dlc/tricky-towers-gem-bricks/',
 '/ru/dlc/tricky-towers-holographic-bricks/',
 '/ru/dlc/tricky-towers-original-soundtrack/']

In [26]:
mm=process_page('https://gg.deals/game/nier-replicant-ver122474487139/')

In [27]:
mm

{'url': 'https://gg.deals/game/nier-replicant-ver122474487139/'}

In [55]:
headers={'x-requested-with':'XMLHttpRequest'}
requests.get('https://gg.deals/ru/games/relations/21937/?type=dlc&offset=4&hideKeyshops=0',headers=headers).url

'https://gg.deals/ru/games/relations/21937/?type=dlc&offset=4&hideKeyshops=0'

In [57]:
soup=BeautifulSoup(requests.get('https://gg.deals/game/tricky-towers/').text, 'lxml')

In [58]:
[x['href'] for x in soup.find_all('a', attrs={'class':'full-link', 'href':True})]

['/ru/redirect/2a8957805c99632e75cf1a2b18826629773e0355/?utm_source=games%2Fsingle',
 '/ru/redirect/083d41787a5cb792387352921b337df85a8bb740/?utm_source=games%2Fsingle',
 '/ru/redirect/78ad462d83243125a2fecc68de4a60c2ab745c5a/?utm_source=games%2Fsingle',
 '/ru/redirect/99a8293924e9b331203da10130580e3ba0569f8c/?utm_source=games%2Fsingle',
 '/ru/redirect/8c03a460f74138ee156aef21c750b64e3e227309/?utm_source=games%2Fsingle',
 '/ru/redirect/c217df75baf6c0896dd8b3e2b60a2489e8d650d5/?utm_source=games%2Fsingle',
 '#',
 '#',
 '#',
 '#',
 '/ru/dlc/tricky-towers-indie-friends-pack/',
 '/ru/dlc/tricky-towers-spirit-animal-pack/',
 '/ru/dlc/tricky-towers-galaxy-bricks/',
 '/ru/dlc/tricky-towers-candy-bricks/',
 '/game/pummel-party/',
 '/game/gang-beasts/',
 '/game/fall-guys-ultimate-knockout/',
 '/game/among-us/',
 '/game/the-jackbox-party-pack-7/',
 '/game/stick-fight-the-game/',
 '/game/overcooked-2/',
 '/game/overcooked-all-you-can-eat/']

In [62]:
[x['href'] for x in soup.find('section', id='game-dlcs').find_all('a', attrs={'class':'full-link', 'href':True})]

['/ru/dlc/tricky-towers-indie-friends-pack/',
 '/ru/dlc/tricky-towers-spirit-animal-pack/',
 '/ru/dlc/tricky-towers-galaxy-bricks/',
 '/ru/dlc/tricky-towers-candy-bricks/']

In [68]:
link_more=soup.find('div', class_='list-show-more').find('a')['data-url']

In [69]:
page = requests.get('https://gg.deals' + link_more,headers=headers)
game_html=page.text
soup = BeautifulSoup(game_html, 'lxml')
soup.find_all('a', attrs={'class':'full-link', 'href':True})

[<a class="full-link" href="/ru/dlc/tricky-towers-gem-bricks/"></a>,
 <a class="full-link" href="/ru/dlc/tricky-towers-holographic-bricks/"></a>,
 <a class="full-link" href="/ru/dlc/tricky-towers-original-soundtrack/"></a>]

In [95]:
s= BeautifulSoup(requests.get('https://gg.deals/game/tricky-towers/').text, 'lxml')

In [96]:
s.find('div', class_='list-show-more').find('a').find("a",data-url=)

<a class="widget-link-more news-list-show-more-link load-more-button" data-url="/ru/games/relations/21937/?type=dlc&amp;offset=4&amp;hideKeyshops=0" href="#">Show more</a>

In [99]:
s= BeautifulSoup(requests.get(mas_of_urls[1]).text, 'lxml')

In [100]:
s.find('div', class_='list-show-more').find('a')

AttributeError: 'NoneType' object has no attribute 'find'

In [163]:
s= BeautifulSoup(requests.get('https://gg.deals/game/tricky-towers/').text, 'lxml')

In [119]:
s.find('div', class_='chart-container')['data-with-keyshops-url']

'/ru/games/chartHistoricalData/21937/?hideKeyshops=0'

In [120]:
page = requests.get('https://gg.deals/ru/games/chartHistoricalData/21937/?hideKeyshops=0',headers=headers)

In [151]:
file= BeautifulSoup(page.text, 'lxml')
d=json.loads(file.text)
len(d['chartData']['deals'])

114

In [160]:
price_history=[]
for el in d['chartData']['deals']:
    tek={"ts":el['x'], "price":el['y'], "shop":el['shop']}
    price_history.append(tek)
price_history

[{'ts': 1470140057000, 'price': 296, 'shop': 'Steam'},
 {'ts': 1470763096000, 'price': 349, 'shop': 'Steam'},
 {'ts': 1474161062000, 'price': 499, 'shop': 'Steam'},
 {'ts': 1477683754000, 'price': 334, 'shop': 'Steam'},
 {'ts': 1478026596000, 'price': 499, 'shop': 'Steam'},
 {'ts': 1479287466000, 'price': 299, 'shop': 'Steam'},
 {'ts': 1479806020000, 'price': 499, 'shop': 'Steam'},
 {'ts': 1480019837000, 'price': 299, 'shop': 'Steam'},
 {'ts': 1480456824000, 'price': 499, 'shop': 'Steam'},
 {'ts': 1482264397000, 'price': 456.89, 'shop': 'WinGameStore'},
 {'ts': 1482640273000, 'price': 249, 'shop': 'Steam'},
 {'ts': 1483386163000, 'price': 499, 'shop': 'Steam'},
 {'ts': 1484245887000, 'price': 449.93, 'shop': 'Humble Store'},
 {'ts': 1484597289000, 'price': 448.1, 'shop': 'WinGameStore'},
 {'ts': 1485158767000, 'price': 869.73, 'shop': 'WinGameStore'},
 {'ts': 1487004600000, 'price': 58.11, 'shop': 'Bundle Stars'},
 {'ts': 1488211200000, 'price': 499, 'shop': 'Steam'},
 {'ts': 149072430

In [156]:
d['chartData']['deals'][0]['x']

1470140057000

In [174]:
def get_price_history(soup):
    adress=soup.find('div', class_='chart-container')['data-with-keyshops-url']
    if adress is None:
        return None
    if adress is not None:
        page=requests.get('https://gg.deals'+adress,headers=headers)
        file= BeautifulSoup(page.text, 'lxml')
        my_dict=json.loads(file.text)['chartData']['deals']
        price_history=[]
        for el in my_dict:
            tek={"ts":el['x'], "price":el['y'], "shop":el['shop']}
            price_history.append(tek)
        return price_history

In [175]:
get_price_history(s)

[{'ts': 1470140057000, 'price': 296, 'shop': 'Steam'},
 {'ts': 1470763096000, 'price': 349, 'shop': 'Steam'},
 {'ts': 1474161062000, 'price': 499, 'shop': 'Steam'},
 {'ts': 1477683754000, 'price': 334, 'shop': 'Steam'},
 {'ts': 1478026596000, 'price': 499, 'shop': 'Steam'},
 {'ts': 1479287466000, 'price': 299, 'shop': 'Steam'},
 {'ts': 1479806020000, 'price': 499, 'shop': 'Steam'},
 {'ts': 1480019837000, 'price': 299, 'shop': 'Steam'},
 {'ts': 1480456824000, 'price': 499, 'shop': 'Steam'},
 {'ts': 1482264397000, 'price': 456.89, 'shop': 'WinGameStore'},
 {'ts': 1482640273000, 'price': 249, 'shop': 'Steam'},
 {'ts': 1483386163000, 'price': 499, 'shop': 'Steam'},
 {'ts': 1484245887000, 'price': 449.93, 'shop': 'Humble Store'},
 {'ts': 1484597289000, 'price': 448.1, 'shop': 'WinGameStore'},
 {'ts': 1485158767000, 'price': 869.73, 'shop': 'WinGameStore'},
 {'ts': 1487004600000, 'price': 58.11, 'shop': 'Bundle Stars'},
 {'ts': 1488211200000, 'price': 499, 'shop': 'Steam'},
 {'ts': 149072430